
 # 📝 Notebook 10: GPT/Qwen/DeepSeek 文本生成比較實戰
 
 **學習目標 (Learning Objectives)**
 - 掌握主流開源 LLM 的載入與推理技巧
 - 理解記憶體優化策略 (4bit/8bit quantization, device mapping)
 - 探索生成參數對輸出品質的影響
 - 建立標準化的性能評估流程

 **涵蓋模型系列 (Model Families)**
 - GPT-2: 英文基礎生成模型
 - Qwen2.5-Instruct: 中英雙語指令模型  
 - DeepSeek-R1-Distill: 推理增強模型

 **適用場景 (Use Cases)**
 - 創意寫作、問答對話、程式生成、文件翻譯


In [ ]:
# %% [code]
# === Shared Cache Bootstrap (English comments only) ===
import os, torch, platform, pathlib, time, psutil
from typing import Dict, List, Optional, Union, Any
import json

# Set up shared cache paths
AI_CACHE_ROOT = os.getenv("AI_CACHE_ROOT", "/mnt/ai/cache")
paths = {
    "HF_HOME": f"{AI_CACHE_ROOT}/hf",
    "TRANSFORMERS_CACHE": f"{AI_CACHE_ROOT}/hf/transformers",
    "HF_DATASETS_CACHE": f"{AI_CACHE_ROOT}/hf/datasets",
    "HUGGINGFACE_HUB_CACHE": f"{AI_CACHE_ROOT}/hf/hub",
    "TORCH_HOME": f"{AI_CACHE_ROOT}/torch",
}

for k, v in paths.items():
    os.environ[k] = v
    pathlib.Path(v).mkdir(parents=True, exist_ok=True)

print(f"[Cache] Root: {AI_CACHE_ROOT}")
print(f"[GPU] Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"[GPU] Device: {torch.cuda.get_device_name(0)}")
    print(
        f"[GPU] Memory: {torch.cuda.get_device_properties(0).total_memory // 1024**3} GB"
    )
else:
    print("[GPU] Using CPU mode")

# System info
print(f"[System] RAM: {psutil.virtual_memory().total // 1024**3} GB")
print(f"[Python] Version: {platform.python_version()}")

In [ ]:
# %% [code]
# === Essential imports for LLM text generation ===
import warnings

warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    pipeline,
    set_seed,
)
from accelerate import init_empty_weights, load_checkpoint_and_dispatch
import torch.nn.functional as F
from datetime import datetime
import gc
import tracemalloc

# Set reproducible seed
set_seed(42)
torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(42)

print("[Setup] Essential imports completed")

In [ ]:
# %% [code]
# === UnifiedModelLoader: Memory-efficient model loading ===
class UnifiedModelLoader:
    """
    Unified loader for GPT-2, Qwen, DeepSeek models with automatic VRAM optimization
    """

    def __init__(self, low_vram_mode: bool = None):
        self.device = "cuda" if torch.cuda.is_available() else "cpu"

        # Auto-detect low VRAM mode based on available memory
        if low_vram_mode is None:
            if torch.cuda.is_available():
                total_vram = torch.cuda.get_device_properties(0).total_memory
                self.low_vram_mode = total_vram < 12 * 1024**3  # Less than 12GB
            else:
                self.low_vram_mode = True
        else:
            self.low_vram_mode = low_vram_mode

        print(f"[Loader] Device: {self.device}, Low-VRAM mode: {self.low_vram_mode}")

    def get_quantization_config(self) -> Optional[BitsAndBytesConfig]:
        """Get 4bit quantization config for memory efficiency"""
        if not self.low_vram_mode or self.device == "cpu":
            return None

        return BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_compute_dtype=torch.float16,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_use_double_quant=True,
        )

    def load_model_and_tokenizer(self, model_id: str) -> tuple:
        """
        Load model and tokenizer with memory optimization

        Args:
            model_id: HuggingFace model identifier

        Returns:
            tuple: (model, tokenizer)
        """
        print(f"[Loading] {model_id}")
        start_time = time.time()

        try:
            # Load tokenizer
            tokenizer = AutoTokenizer.from_pretrained(
                model_id,
                trust_remote_code=True,
                cache_dir=os.environ["TRANSFORMERS_CACHE"],
            )

            # Ensure pad token exists
            if tokenizer.pad_token is None:
                tokenizer.pad_token = tokenizer.eos_token

            # Prepare model loading kwargs
            model_kwargs = {
                "trust_remote_code": True,
                "cache_dir": os.environ["TRANSFORMERS_CACHE"],
                "torch_dtype": (
                    torch.float16 if self.device == "cuda" else torch.float32
                ),
            }

            # Add quantization config if low VRAM mode
            quantization_config = self.get_quantization_config()
            if quantization_config:
                model_kwargs["quantization_config"] = quantization_config
                print(f"[Loading] Using 4bit quantization")
            else:
                model_kwargs["device_map"] = "auto" if self.device == "cuda" else None

            # Load model
            model = AutoModelForCausalLM.from_pretrained(model_id, **model_kwargs)

            # Move to device if not using device_map
            if not quantization_config and self.device == "cuda":
                model = model.to(self.device)

            load_time = time.time() - start_time

            # Calculate model size
            param_count = sum(p.numel() for p in model.parameters())
            param_size_mb = param_count * 4 / 1024**2  # Assuming float32

            print(f"[Loaded] {model_id}")
            print(f"[Stats] Parameters: {param_count:,} ({param_size_mb:.1f} MB)")
            print(f"[Stats] Load time: {load_time:.2f}s")

            return model, tokenizer

        except Exception as e:
            print(f"[Error] Failed to load {model_id}: {str(e)}")
            return None, None

    def generate_text(
        self,
        model,
        tokenizer,
        prompt: str,
        max_new_tokens: int = 256,
        temperature: float = 0.8,
        top_k: int = 50,
        top_p: float = 0.9,
        repetition_penalty: float = 1.1,
        do_sample: bool = True,
    ) -> Dict[str, Any]:
        """
        Generate text with detailed statistics

        Returns:
            dict: Generated text, stats, and metadata
        """
        start_time = time.time()

        # Encode input
        inputs = tokenizer.encode(prompt, return_tensors="pt")
        if self.device == "cuda" and inputs.device != model.device:
            inputs = inputs.to(model.device)

        input_length = inputs.shape[1]

        # Generation parameters
        gen_kwargs = {
            "max_new_tokens": max_new_tokens,
            "temperature": temperature,
            "top_k": top_k,
            "top_p": top_p,
            "repetition_penalty": repetition_penalty,
            "do_sample": do_sample,
            "pad_token_id": tokenizer.eos_token_id,
        }

        # Track memory before generation
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
            mem_before = torch.cuda.memory_allocated()

        # Generate
        with torch.no_grad():
            outputs = model.generate(inputs, **gen_kwargs)

        # Decode output
        generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
        new_text = generated_text[len(prompt) :]

        # Calculate stats
        generation_time = time.time() - start_time
        output_length = outputs.shape[1] - input_length
        tokens_per_sec = output_length / generation_time if generation_time > 0 else 0

        if torch.cuda.is_available():
            mem_after = torch.cuda.memory_allocated()
            peak_memory_mb = (mem_after - mem_before) / 1024**2
        else:
            peak_memory_mb = 0

        return {
            "generated_text": new_text,
            "full_text": generated_text,
            "stats": {
                "input_tokens": input_length,
                "output_tokens": output_length,
                "total_tokens": outputs.shape[1],
                "generation_time": generation_time,
                "tokens_per_sec": tokens_per_sec,
                "peak_memory_mb": peak_memory_mb,
            },
            "parameters": gen_kwargs,
        }


# Initialize loader
loader = UnifiedModelLoader()
print("[Setup] UnifiedModelLoader ready")


 ## 🤖 模型定義與載入策略
 
 **記憶體需求評估 (VRAM Requirements)**
 - GPT-2 Small (124M): ~0.5GB
 - GPT-2 Medium (355M): ~1.4GB  
 - GPT-2 Large (774M): ~3.1GB
 - Qwen2.5-7B-Instruct (4bit): ~4.5GB
 - DeepSeek-R1-Distill-Qwen-7B (4bit): ~4.5GB

 **量化策略 (Quantization Strategy)**
 - <8GB VRAM: 強制 4bit 量化
 - 8-16GB VRAM: 可選 4bit/8bit
 - >16GB VRAM: 原生 float16


In [ ]:
# %% [code]
# === Model configurations and loading ===
MODEL_CONFIGS = {
    "gpt2-small": {
        "model_id": "gpt2",
        "description": "GPT-2 Small (124M) - 英文基礎生成",
        "vram_requirement": 0.5,
        "strengths": ["輕量快速", "英文流暢", "創意寫作"],
    },
    "gpt2-medium": {
        "model_id": "gpt2-medium",
        "description": "GPT-2 Medium (355M) - 平衡性能與品質",
        "vram_requirement": 1.4,
        "strengths": ["中等規模", "較佳連貫性", "詩詞創作"],
    },
    "qwen2.5-7b-instruct": {
        "model_id": "Qwen/Qwen2.5-7B-Instruct",
        "description": "Qwen2.5-7B-Instruct - 中英雙語指令模型",
        "vram_requirement": 4.5,  # with 4bit quantization
        "strengths": ["中文優秀", "指令遵循", "多語支持"],
    },
    "deepseek-r1-distill": {
        "model_id": "deepseek-ai/deepseek-r1-distill-qwen-7b",
        "description": "DeepSeek-R1-Distill-Qwen-7B - 推理增強模型",
        "vram_requirement": 4.5,  # with 4bit quantization
        "strengths": ["邏輯推理", "數學問題", "思維鏈"],
    },
}


def display_model_info():
    """Display available models and their characteristics"""
    print("=" * 60)
    print("🤖 可用模型總覽 (Available Models)")
    print("=" * 60)

    for key, config in MODEL_CONFIGS.items():
        print(f"\n📋 {key.upper()}")
        print(f"   ID: {config['model_id']}")
        print(f"   描述: {config['description']}")
        print(f"   VRAM: ~{config['vram_requirement']}GB")
        print(f"   優勢: {', '.join(config['strengths'])}")

    print("\n" + "=" * 60)


display_model_info()

In [ ]:
# %% [code]
# === Load and test GPT-2 models ===
print("🔄 正在載入 GPT-2 系列模型 (Loading GPT-2 models)")

# Load GPT-2 Small for quick testing
gpt2_model, gpt2_tokenizer = loader.load_model_and_tokenizer("gpt2")

if gpt2_model is not None:
    # Test English creative writing
    english_prompts = [
        "Once upon a time in a magical forest,",
        "The future of artificial intelligence will be",
        "In the year 2030, technology will have",
    ]

    print("\n📝 GPT-2 英文創意生成測試")
    print("-" * 40)

    for i, prompt in enumerate(english_prompts, 1):
        print(f"\n💭 Prompt {i}: {prompt}")

        result = loader.generate_text(
            gpt2_model,
            gpt2_tokenizer,
            prompt,
            max_new_tokens=100,
            temperature=0.8,
            top_p=0.9,
        )

        print(f"📄 Generated: {result['generated_text'][:200]}...")
        print(f"⚡ Speed: {result['stats']['tokens_per_sec']:.1f} tokens/sec")
        print(f"💾 Memory: {result['stats']['peak_memory_mb']:.1f} MB")

else:
    print("❌ GPT-2 載入失敗")

In [ ]:
# %% [code]
# === Load and test Qwen2.5-7B-Instruct ===
print("\n🔄 正在載入 Qwen2.5-7B-Instruct 模型")

# Free up memory from previous model if needed
if "gpt2_model" in locals() and gpt2_model is not None:
    del gpt2_model, gpt2_tokenizer
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

qwen_model, qwen_tokenizer = loader.load_model_and_tokenizer("Qwen/Qwen2.5-7B-Instruct")

if qwen_model is not None:
    # Test Chinese and English instruction following
    instruction_prompts = [
        "請用繁體中文寫一首關於春天的短詩。",
        "解釋什麼是機器學習，用簡單易懂的方式。",
        "Write a Python function to calculate fibonacci numbers.",
        "Translate this to Chinese: 'Artificial intelligence is transforming our world.'",
    ]

    print("\n📝 Qwen2.5 中英雙語指令測試")
    print("-" * 40)

    for i, prompt in enumerate(instruction_prompts, 1):
        print(f"\n💭 Instruction {i}: {prompt}")

        result = loader.generate_text(
            qwen_model,
            qwen_tokenizer,
            prompt,
            max_new_tokens=150,
            temperature=0.7,
            top_p=0.8,
            repetition_penalty=1.1,
        )

        print(f"📄 Response: {result['generated_text']}")
        print(f"⚡ Speed: {result['stats']['tokens_per_sec']:.1f} tokens/sec")
        print(f"💾 Memory: {result['stats']['peak_memory_mb']:.1f} MB")
        print("-" * 40)

else:
    print("❌ Qwen2.5 載入失敗")

In [ ]:
# %% [code]
# === Load and test DeepSeek-R1-Distill ===
print("\n🔄 正在載入 DeepSeek-R1-Distill 模型")

# Free up memory from Qwen if needed
if "qwen_model" in locals() and qwen_model is not None:
    del qwen_model, qwen_tokenizer
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

deepseek_model, deepseek_tokenizer = loader.load_model_and_tokenizer(
    "deepseek-ai/deepseek-r1-distill-qwen-7b"
)

if deepseek_model is not None:
    # Test reasoning and math problems
    reasoning_prompts = [
        "Solve this step by step: If a train travels 120 km in 1.5 hours, what is its average speed?",
        "邏輯推理：如果所有的貓都有尾巴，而小花是一隻貓，那麼小花有尾巴嗎？請說明原因。",
        "Write Python code to find the prime factors of 84, with explanations.",
        "如果今天是星期三，那麼100天後是星期幾？請詳細計算。",
    ]

    print("\n📝 DeepSeek-R1-Distill 推理增強測試")
    print("-" * 40)

    for i, prompt in enumerate(reasoning_prompts, 1):
        print(f"\n💭 Reasoning Task {i}: {prompt}")

        result = loader.generate_text(
            deepseek_model,
            deepseek_tokenizer,
            prompt,
            max_new_tokens=200,
            temperature=0.3,  # Lower temperature for reasoning
            top_p=0.8,
            repetition_penalty=1.05,
        )

        print(f"📄 Solution: {result['generated_text']}")
        print(f"⚡ Speed: {result['stats']['tokens_per_sec']:.1f} tokens/sec")
        print(f"💾 Memory: {result['stats']['peak_memory_mb']:.1f} MB")
        print("-" * 40)

else:
    print("❌ DeepSeek-R1-Distill 載入失敗")


 ## 🎛️ 生成參數調優實驗
 
 **核心參數說明 (Parameter Explanation)**
 - **Temperature**: 控制隨機性 (0.1=保守, 1.0=創意, 2.0=極富創意)
 - **Top-k**: 候選詞數量限制 (10=嚴格, 50=平衡, 100=寬鬆)
 - **Top-p**: 累積機率閾值 (0.8=保守, 0.9=平衡, 0.95=創意)
 - **Repetition Penalty**: 重複懲罰 (1.0=無懲罰, 1.1=輕微, 1.3=嚴格)


In [ ]:
# %% [code]
# === Parameter tuning experiments ===
def parameter_experiment(model, tokenizer, prompt: str, model_name: str):
    """
    Conduct systematic parameter tuning experiments
    """
    print(f"\n🧪 {model_name} 參數調優實驗")
    print(f"📝 測試提示: {prompt}")
    print("=" * 60)

    # Parameter grid for testing
    param_configs = [
        {
            "name": "保守設定",
            "temp": 0.3,
            "top_k": 20,
            "top_p": 0.8,
            "rep_penalty": 1.1,
        },
        {
            "name": "平衡設定",
            "temp": 0.7,
            "top_k": 50,
            "top_p": 0.9,
            "rep_penalty": 1.05,
        },
        {
            "name": "創意設定",
            "temp": 1.0,
            "top_k": 100,
            "top_p": 0.95,
            "rep_penalty": 1.0,
        },
        {
            "name": "極端創意",
            "temp": 1.5,
            "top_k": 0,
            "top_p": 0.98,
            "rep_penalty": 0.95,
        },
    ]

    results = []

    for config in param_configs:
        print(
            f"\n🎯 {config['name']} (T={config['temp']}, k={config['top_k']}, p={config['top_p']}, rp={config['rep_penalty']})"
        )

        result = loader.generate_text(
            model,
            tokenizer,
            prompt,
            max_new_tokens=120,
            temperature=config["temp"],
            top_k=config["top_k"] if config["top_k"] > 0 else None,
            top_p=config["top_p"],
            repetition_penalty=config["rep_penalty"],
        )

        print(f"📄 輸出: {result['generated_text'][:150]}...")
        print(f"⚡ 速度: {result['stats']['tokens_per_sec']:.1f} tokens/sec")

        results.append(
            {
                "config": config["name"],
                "output": result["generated_text"],
                "speed": result["stats"]["tokens_per_sec"],
                "length": result["stats"]["output_tokens"],
            }
        )

        print("-" * 40)

    return results


# Run parameter experiments if model is available
if "deepseek_model" in locals() and deepseek_model is not None:
    test_prompt = "寫一個關於人工智慧的短故事："
    param_results = parameter_experiment(
        deepseek_model, deepseek_tokenizer, test_prompt, "DeepSeek-R1-Distill"
    )

    # Create comparison DataFrame
    param_df = pd.DataFrame(param_results)
    print("\n📊 參數調優結果總覽")
    print(param_df[["config", "speed", "length"]])

In [ ]:
# %% [code]
# === Multi-scenario application demonstrations ===
print("\n🎭 多場景應用示範 (Multi-scenario Applications)")
print("=" * 60)

# Define application scenarios
scenarios = {
    "程式碼生成": {
        "prompt": "Write a Python function to merge two sorted lists:",
        "params": {"temperature": 0.2, "top_p": 0.8, "max_new_tokens": 200},
    },
    "文件摘要": {
        "prompt": "請摘要以下內容的重點：人工智慧是一門研究如何讓機器模擬人類智慧的學科，包括機器學習、深度學習、自然語言處理等多個分支領域。",
        "params": {"temperature": 0.3, "top_p": 0.8, "max_new_tokens": 100},
    },
    "創意寫作": {
        "prompt": "在一個科技高度發達的未來城市裡，",
        "params": {"temperature": 0.9, "top_p": 0.95, "max_new_tokens": 150},
    },
    "翻譯任務": {
        "prompt": "Translate to Traditional Chinese: 'Machine learning algorithms can process vast amounts of data to identify patterns and make predictions.'",
        "params": {"temperature": 0.1, "top_p": 0.8, "max_new_tokens": 100},
    },
}

application_results = {}

# Test scenarios with available model
current_model = None
current_tokenizer = None
current_model_name = "Unknown"

# Determine which model is currently loaded
if "deepseek_model" in locals() and deepseek_model is not None:
    current_model = deepseek_model
    current_tokenizer = deepseek_tokenizer
    current_model_name = "DeepSeek-R1-Distill"
elif "qwen_model" in locals() and qwen_model is not None:
    current_model = qwen_model
    current_tokenizer = qwen_tokenizer
    current_model_name = "Qwen2.5-7B-Instruct"
elif "gpt2_model" in locals() and gpt2_model is not None:
    current_model = gpt2_model
    current_tokenizer = gpt2_tokenizer
    current_model_name = "GPT-2"

if current_model is not None:
    print(f"🤖 使用模型: {current_model_name}")

    for scenario_name, scenario_config in scenarios.items():
        print(f"\n🎯 場景: {scenario_name}")
        print(f"💭 提示: {scenario_config['prompt']}")

        result = loader.generate_text(
            current_model,
            current_tokenizer,
            scenario_config["prompt"],
            **scenario_config["params"],
        )

        print(f"📄 結果: {result['generated_text']}")
        print(f"⚡ 性能: {result['stats']['tokens_per_sec']:.1f} tokens/sec")

        application_results[scenario_name] = result
        print("-" * 40)
else:
    print("❌ 無可用模型進行應用測試")

In [ ]:
# %% [code]
# === Performance benchmarking and comparison ===
class ModelBenchmark:
    """
    Comprehensive model performance benchmarking
    """

    def __init__(self):
        self.results = []

    def benchmark_model(self, model, tokenizer, model_name: str, num_runs: int = 3):
        """
        Benchmark a model across multiple metrics
        """
        print(f"\n📊 {model_name} 性能基準測試")
        print("-" * 40)

        # Standard test prompts
        test_prompts = [
            "The future of artificial intelligence",
            "解釋機器學習的基本概念",
            "Write code to calculate factorial",
        ]

        # Collect metrics across multiple runs
        all_metrics = []

        for run in range(num_runs):
            run_metrics = []

            for prompt in test_prompts:
                # Warm up run (not counted)
                if run == 0:
                    _ = loader.generate_text(
                        model, tokenizer, prompt, max_new_tokens=50
                    )

                # Actual benchmark run
                start_time = time.time()
                result = loader.generate_text(
                    model,
                    tokenizer,
                    prompt,
                    max_new_tokens=100,
                    temperature=0.7,
                    top_p=0.9,
                )

                run_metrics.append(
                    {
                        "prompt": prompt[:30] + "...",
                        "tokens_per_sec": result["stats"]["tokens_per_sec"],
                        "generation_time": result["stats"]["generation_time"],
                        "output_tokens": result["stats"]["output_tokens"],
                        "peak_memory_mb": result["stats"]["peak_memory_mb"],
                    }
                )

            all_metrics.extend(run_metrics)

        # Calculate average metrics
        avg_metrics = {
            "model_name": model_name,
            "avg_tokens_per_sec": np.mean([m["tokens_per_sec"] for m in all_metrics]),
            "avg_generation_time": np.mean([m["generation_time"] for m in all_metrics]),
            "avg_output_tokens": np.mean([m["output_tokens"] for m in all_metrics]),
            "avg_peak_memory_mb": np.mean([m["peak_memory_mb"] for m in all_metrics]),
            "std_tokens_per_sec": np.std([m["tokens_per_sec"] for m in all_metrics]),
        }

        # Print results
        print(
            f"🏃 平均速度: {avg_metrics['avg_tokens_per_sec']:.1f} ± {avg_metrics['std_tokens_per_sec']:.1f} tokens/sec"
        )
        print(f"⏱️ 平均生成時間: {avg_metrics['avg_generation_time']:.2f}s")
        print(f"💾 平均記憶體使用: {avg_metrics['avg_peak_memory_mb']:.1f} MB")
        print(f"📝 平均輸出長度: {avg_metrics['avg_output_tokens']:.0f} tokens")

        self.results.append(avg_metrics)
        return avg_metrics

    def compare_models(self):
        """Generate comparison report"""
        if len(self.results) < 2:
            print("⚠️ 需要至少兩個模型進行比較")
            return

        df = pd.DataFrame(self.results)
        print("\n📈 模型性能比較表")
        print("=" * 80)
        print(
            df[["model_name", "avg_tokens_per_sec", "avg_peak_memory_mb"]].to_string(
                index=False
            )
        )

        # Find best performers
        fastest = df.loc[df["avg_tokens_per_sec"].idxmax()]
        most_efficient = df.loc[df["avg_peak_memory_mb"].idxmin()]

        print(
            f"\n🏆 最快模型: {fastest['model_name']} ({fastest['avg_tokens_per_sec']:.1f} tokens/sec)"
        )
        print(
            f"💡 最節省記憶體: {most_efficient['model_name']} ({most_efficient['avg_peak_memory_mb']:.1f} MB)"
        )


# Initialize benchmark
benchmark = ModelBenchmark()

# Benchmark currently loaded model
if current_model is not None:
    benchmark.benchmark_model(current_model, current_tokenizer, current_model_name)

 ## ⚡ 記憶體與性能優化技巧
 
 **低 VRAM 優化策略 (Low-VRAM Optimization)**
 1. **量化 (Quantization)**: 4bit/8bit 減少 50-75% 記憶體使用
 2. **CPU Offload**: 將部分層移至 CPU 記憶體
 3. **Gradient Checkpointing**: 犧牲計算換取記憶體
 4. **Dynamic Batching**: 根據序列長度動態調整批次大小

 **生成速度優化 (Speed Optimization)**
 1. **KV Cache**: 避免重複計算注意力
 2. **Beam Search 替代**: 使用 Top-k/Top-p 採樣
 3. **Early Stopping**: 設定合理的最大長度
 4. **Model Compilation**: 使用 torch.compile (PyTorch 2.0+)


In [ ]:
# %% [code]
# === Memory and performance optimization demonstrations ===
def demonstrate_optimization_techniques():
    """
    Show various optimization techniques for different scenarios
    """
    print("\n🔧 記憶體與性能優化示範")
    print("=" * 60)

    optimization_tips = {
        "4bit 量化": {
            "description": "減少 75% 記憶體使用，輕微精度損失",
            "code": """
# 4bit quantization config
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4"
)
model = AutoModelForCausalLM.from_pretrained(
    model_id, quantization_config=quantization_config
)""",
        },
        "CPU Offload": {
            "description": "將未使用的層移至 CPU，節省 VRAM",
            "code": """
# CPU offload with device_map
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    offload_folder="./offload"
)""",
        },
        "動態批次": {
            "description": "根據序列長度調整批次大小",
            "code": """
# Dynamic batching based on sequence length
def get_batch_size(seq_length):
    if seq_length < 512:
        return 8
    elif seq_length < 1024:
        return 4
    else:
        return 2""",
        },
        "生成優化": {
            "description": "使用 KV cache 和早停策略",
            "code": """
# Optimized generation
outputs = model.generate(
    inputs,
    max_new_tokens=256,
    use_cache=True,  # Enable KV cache
    do_sample=True,
    early_stopping=True,
    pad_token_id=tokenizer.eos_token_id
)""",
        },
    }

    for technique, info in optimization_tips.items():
        print(f"\n💡 {technique}")
        print(f"   📝 說明: {info['description']}")
        print(f"   💻 程式碼:")
        print("   " + "\n   ".join(info["code"].split("\n")))
        print("-" * 40)


demonstrate_optimization_techniques()

In [ ]:
# %% [code]
# === Practical usage guidelines and best practices ===
def generate_usage_guidelines():
    """
    Provide practical guidelines for model selection and usage
    """
    guidelines = {
        "模型選擇建議": {
            "輕量任務 (<2GB VRAM)": ["GPT-2 Small/Medium", "適合快速原型和測試"],
            "中文任務 (4-8GB VRAM)": [
                "Qwen2.5-7B-Instruct (4bit)",
                "優秀的中文理解和生成",
            ],
            "推理任務 (4-8GB VRAM)": [
                "DeepSeek-R1-Distill (4bit)",
                "邏輯推理和數學問題",
            ],
            "高品質生成 (>12GB VRAM)": ["Qwen2.5-14B/32B", "最佳生成品質"],
        },
        "參數調優指南": {
            "創意寫作": "temperature=0.8-1.2, top_p=0.9-0.95",
            "技術文檔": "temperature=0.1-0.3, top_p=0.8-0.9",
            "程式碼生成": "temperature=0.1-0.2, top_p=0.8",
            "對話系統": "temperature=0.6-0.8, top_p=0.9",
        },
        "常見問題解決": {
            "記憶體不足": "使用 4bit 量化 + CPU offload",
            "生成速度慢": "減少 max_new_tokens + 使用 top_k 採樣",
            "重複文本": "增加 repetition_penalty (1.1-1.3)",
            "輸出不穩定": "降低 temperature + 設定隨機種子",
        },
    }

    print("\n📚 實用指南與最佳實踐")
    print("=" * 60)

    for category, items in guidelines.items():
        print(f"\n🎯 {category}")
        for key, value in items.items():
            if isinstance(value, list):
                print(f"   • {key}: {', '.join(value)}")
            else:
                print(f"   • {key}: {value}")
        print("-" * 40)


generate_usage_guidelines()

In [ ]:
# %% [code]
# === Final smoke test and validation ===
def run_smoke_test():
    """
    Comprehensive smoke test to validate all functionality
    """
    print("\n🧪 驗收測試 (Smoke Test)")
    print("=" * 50)

    test_results = {
        "shared_cache_setup": False,
        "model_loading": False,
        "text_generation": False,
        "parameter_tuning": False,
        "memory_monitoring": False,
    }

    # Test 1: Shared cache setup
    try:
        cache_exists = all(os.path.exists(path) for path in paths.values())
        test_results["shared_cache_setup"] = cache_exists
        print(f"✅ 共享快取設定: {'通過' if cache_exists else '失敗'}")
    except Exception as e:
        print(f"❌ 共享快取設定失敗: {e}")

    # Test 2: Model loading
    try:
        test_model, test_tokenizer = loader.load_model_and_tokenizer("gpt2")
        test_results["model_loading"] = test_model is not None
        print(f"✅ 模型載入: {'通過' if test_model is not None else '失敗'}")

        if test_model is not None:
            # Test 3: Text generation
            try:
                result = loader.generate_text(
                    test_model, test_tokenizer, "Hello, world!", max_new_tokens=10
                )
                test_results["text_generation"] = len(result["generated_text"]) > 0
                print(
                    f"✅ 文本生成: {'通過' if len(result['generated_text']) > 0 else '失敗'}"
                )

                # Test 4: Parameter tuning
                try:
                    result_low_temp = loader.generate_text(
                        test_model,
                        test_tokenizer,
                        "Hello, world!",
                        max_new_tokens=10,
                        temperature=0.1,
                    )
                    result_high_temp = loader.generate_text(
                        test_model,
                        test_tokenizer,
                        "Hello, world!",
                        max_new_tokens=10,
                        temperature=1.5,
                    )
                    test_results["parameter_tuning"] = True
                    print("✅ 參數調優: 通過")
                except Exception as e:
                    print(f"❌ 參數調優失敗: {e}")

                # Test 5: Memory monitoring
                try:
                    memory_tracked = result["stats"]["peak_memory_mb"] >= 0
                    test_results["memory_monitoring"] = memory_tracked
                    print(f"✅ 記憶體監控: {'通過' if memory_tracked else '失敗'}")
                except Exception as e:
                    print(f"❌ 記憶體監控失敗: {e}")

            except Exception as e:
                print(f"❌ 文本生成失敗: {e}")

        # Cleanup test model
        if test_model is not None:
            del test_model, test_tokenizer
            gc.collect()
            if torch.cuda.is_available():
                torch.cuda.empty_cache()

    except Exception as e:
        print(f"❌ 模型載入失敗: {e}")

    # Summary
    passed_tests = sum(test_results.values())
    total_tests = len(test_results)

    print(f"\n📊 測試結果: {passed_tests}/{total_tests} 通過")
    if passed_tests == total_tests:
        print("🎉 所有測試通過！系統運行正常")
    else:
        print("⚠️ 部分測試失敗，請檢查錯誤訊息")

    return test_results


# Run smoke test
smoke_test_results = run_smoke_test()

 ## 📋 章節總結與下一步建議
 
 ### ✅ 完成項目 (Completed Items)
 1. **統一模型載入器** - 支援 GPT-2、Qwen2.5、DeepSeek-R1-Distill 三大系列
 2. **記憶體優化策略** - 4bit 量化、device mapping、CPU offload
 3. **生成參數調優** - temperature、top-k、top-p、repetition penalty 實驗
 4. **多場景應用** - 程式碼生成、文件摘要、創意寫作、翻譯任務
 5. **性能基準測試** - 速度、記憶體使用量、輸出品質評估
 
 ### 🧠 核心概念 (Core Concepts)
 - **Transformer 解碼策略**: Greedy vs Sampling vs Beam Search
 - **量化技術**: BitsAndBytesConfig 4bit/8bit 量化原理
 - **記憶體管理**: VRAM 監控與 CPU offload 策略
 - **參數調優**: 創意性與一致性的平衡
 - **模型特性**: 不同模型的優勢與適用場景
 
 ### ⚠️ 常見陷阱 (Common Pitfalls)
 - **VRAM 溢出**: 未正確設定量化或 device mapping
 - **重複文本**: repetition_penalty 設定過低
 - **生成速度慢**: max_new_tokens 過大或未使用 KV cache
 - **中文支援**: tokenizer 不支援中文或分詞錯誤
 - **記憶體洩漏**: 未正確清理舊模型或快取
 
 ### 🚀 下一步建議 (Next Steps)
 1. **指令調優 (nb11)**: 學習 Alpaca/Dolly 資料格式與微調流程
 2. **LLM 評估 (nb12)**: 實作 perplexity、ROUGE、BLEU 自動評估
 3. **Function Calling (nb13)**: 整合工具調用與 Agent 能力
 4. **RAG 系統 (nb26)**: 結合檢索與生成的問答系統


In [ ]:
# %% [code]
# === Generate next steps and recommendations ===
def generate_next_steps():
    """
    Provide specific recommendations for next learning steps
    """
    next_steps = {
        "立即實踐 (Immediate Practice)": [
            "嘗試不同的中文提示詞，觀察模型的理解能力",
            "實驗更多參數組合，找到最適合你任務的設定",
            "測試模型在特定領域（如醫療、法律、金融）的表現",
            "建立你自己的提示詞模板庫",
        ],
        "深化學習 (Deep Learning)": [
            "研究 Transformer 架構的內部機制",
            "了解不同量化方法的精度與速度權衡",
            "學習模型並行與分散式推理技術",
            "探索新興的長文本處理技術",
        ],
        "實際應用 (Practical Applications)": [
            "建構特定領域的聊天機器人",
            "開發程式碼自動生成與除錯工具",
            "實作多語言翻譯與在地化系統",
            "設計創意寫作輔助應用",
        ],
        "技術提升 (Technical Advancement)": [
            "nb11: 指令調優與資料準備技巧",
            "nb12: 全面的 LLM 評估方法論",
            "nb13: Function Calling 與工具整合",
            "nb26: RAG 檢索增強生成系統",
        ],
    }

    print("\n🎯 學習路徑建議")
    print("=" * 50)

    for category, recommendations in next_steps.items():
        print(f"\n📌 {category}")
        for i, rec in enumerate(recommendations, 1):
            print(f"   {i}. {rec}")
        print("-" * 40)

    # Priority recommendations
    print("\n⭐ 優先建議")
    print("-" * 20)
    print("1. 如果想深入微調: 先完成 nb11 (指令調優)")
    print("2. 如果想建構應用: 先完成 nb13 (Function Calling)")
    print("3. 如果想評估模型: 先完成 nb12 (LLM 評估)")
    print("4. 如果想建構 RAG: 先完成 nb26 (RAG 基礎)")


generate_next_steps()

print("\n" + "=" * 60)
print("🎓 Notebook 10 完成！")
print("📖 你已掌握主流開源 LLM 的載入、調優與應用技巧")
print("🚀 準備好進入下一個階段的學習了！")
print("=" * 60)

In [ ]:
# === Quick validation test ===
def quick_validation():
    """5-line smoke test for immediate verification"""
    cache_ok = os.path.exists(os.environ["TRANSFORMERS_CACHE"])
    model, tokenizer = loader.load_model_and_tokenizer("gpt2")
    if model:
        result = loader.generate_text(model, tokenizer, "Test", max_new_tokens=5)
    print(
        f"✅ Cache: {cache_ok}, Model: {model is not None}, Generation: {'✅' if model and len(result['generated_text']) > 0 else '❌'}"
    )
    return cache_ok and model is not None


quick_validation()

## 本章小結

### ✅ 完成項目 (Completed Items)
- **統一模型載入系統** - 支援 GPT-2、Qwen2.5、DeepSeek-R1-Distill 與自動記憶體優化
- **低 VRAM 友善策略** - 4bit 量化、CPU offload、device mapping 完整實作
- **參數調優實驗室** - 系統化測試 temperature、top-k、top-p、repetition penalty 影響
- **多場景應用展示** - 程式碼生成、創意寫作、翻譯、摘要等實用案例
- **性能基準測試** - 速度、記憶體、品質的標準化評估流程

### 🧠 核心原理要點 (Core Concepts)
- **Transformer 解碼策略**: 貪婪搜尋 vs 採樣 vs Beam Search 的適用時機
- **量化技術原理**: BitsAndBytesConfig 如何實現 75% 記憶體節省
- **生成品質控制**: 創意性 (temperature) 與一致性 (repetition penalty) 的平衡
- **記憶體管理**: VRAM 監控、CPU offload、梯度檢查點的組合策略
- **模型特性分析**: 不同模型系列的優勢與適用場景識別

### 🚀 下一步建議 (Next Steps)
1. **如果想深入微調技術** → 優先學習 `nb11_instruction_tuning_demo.ipynb`
2. **如果想建構實用應用** → 優先學習 `nb13_function_calling_tools.ipynb`  
3. **如果想系統性評估** → 優先學習 `nb12_llm_evaluation_metrics.ipynb`
4. **如果想整合檢索系統** → 優先學習 `nb26_rag_basic_faiss.ipynb`

**技術深化方向**: 探索 KV cache 優化、模型並行推理、長文本處理技術
**應用拓展方向**: 特定領域聊天機器人、程式碼助手、多語言翻譯系統

---

這個 notebook 為你建立了堅實的 LLM 應用基礎，涵蓋了從模型載入到性能優化的完整流程。你現在具備了選擇適合的模型、調優生成參數、處理記憶體限制的實戰能力！